# TME 3.1 : Greedy Profile Motif Search 


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons d'abord implémenter l'algorithm Greedy Profile Motif Search.


1\. Nous allons réutiliser les fonctions du precedent pour générer `t` séquences artificielles de taille `n`, et implanter dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement. Nous allons faire varier le motifs dans 50% de cas.

In [57]:
import random
import numpy as np
import math

nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence
f= 0.5 #frequence de variation du motif.


In [58]:

def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n:int, upper=False):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront minuscule, False majuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    for i in range(n):
        sequence += random.choice(nuc) if upper else random.choice(nuc).lower()
    return sequence

def modifierMotif(motif:str, nbpos:int,  upper=False):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront minuscule, False majuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)
    for i in range(nbpos):
        motifM[random.randint(0,len(motifM)-1)]= random.choice(nuc)
    
    return "".join(motifM).upper() if upper else "".join(motifM).lower()
#tester modifMotif
print (modifierMotif("acg", 2))

def implantMotifVar(k, v, t, n, f):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée n : longueur de séquences 
    entrée t : nombre des séquences
    entrée f : frequence de variation du motif.
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, 
    pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    
    motif = generateRandomSequence(k)
    print(motif)
    sequences = [generateRandomSequence(n - k) for i in range(n)]

    for i in range(k):
        newmotif = motif
        if (random.random() < f):
            newmotif = modifierMotif(motif, v)
        sequences[i] = insertMotif(sequences[i],newmotif , random.randint(0, n - k))
    return sequences
   # return [insertMotif(generateRandomSequence(t - k),modifierMotif(motif, v), random.randint(0, t - k)) for i in range (n)]


adn = implantMotifVar(k, v, t, n, f)
print (adn)

adn  = [s.upper() for s in adn]
print (adn)

aag
agttt
['cggaaagttg', 'ttaagttttt', 'gaatttttgt', 'gagtttgaaa', 'gttagagttt', 'gtttg', 'taacg', 'cgggt', 'tacgc', 'gctgc']
['CGGAAAGTTG', 'TTAAGTTTTT', 'GAATTTTTGT', 'GAGTTTGAAA', 'GTTAGAGTTT', 'GTTTG', 'TAACG', 'CGGGT', 'TACGC', 'GCTGC']


2\. Faire une fonction pour sélectionner des positions de départ aléatoirement `s = (s1, …,st)`.


In [59]:
#creating vector s
#Get t random positions from 0 to n - k
def generateRandomS(sequences, k):
    """
    Génère un vecteur de position aléatoires
    entrée sequences: matrice de dimension txn contenant les sequences
    entrée k: taille du motif
    sortie s: vecteur de position aléatoires, une position par séquence
    REMARQUE les positions doivent être inférieur à n-k, ou n est la taille de sequence
    """
    s = []
    for i in range (len(sequences)) :
        n = len(sequences[i])
        if (n - k >= 0):
            s.append(random.randint(0, n - k ))

    return s
s = generateRandomS(adn, k)
print (s)


[5, 2, 3, 1, 5, 0, 0, 0, 0, 0]


3\. Extraire les motifs en utilisant le vecteur `s`, et construire un profile (matrice de fréquence). 


In [60]:
#extract sequences from seqs
def extractSeqs(s, seqs, k):
    """
    Extraire les motifs des séquences à l'aide de positions s
    entrée s: vecteur contenant les positions de départs
    entrée seqs: matrice de dimension txn contenant les séquences
    entrée k: taille du motif
    sortie motifs: liste de motifs de taille k extrait des séquences
    """
    motifs = []    
    for index in range(len(seqs)):
            motifs.append(seqs[index][s[index]: s[index] + k])
    return motifs
        
motifs = extractSeqs(s, adn, k)
print (motifs)


#Construire un profile = matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension k x |nuc|
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    q = len(nuc)
    PWM = np.zeros((q, k))
    for s in motifs :
        for j in range(k) :
            l = s[j]
            if l.upper() == 'A':
                PWM[0][j] += 1
            elif l.upper() == 'C':
                PWM[1][j] += 1
            elif l.upper() == 'G':
                PWM[2][j] += 1
            elif l.upper() == 'T':
                PWM[3][j] += 1
  
    return PWM

MF = profile(motifs, k, nuc)
print(MF)

['AGTTG', 'AAGTT', 'TTTTT', 'AGTTT', 'AGTTT', 'GTTTG', 'TAACG', 'CGGGT', 'TACGC', 'GCTGC']
[[4. 3. 1. 0. 0.]
 [1. 1. 1. 1. 2.]
 [2. 4. 2. 3. 3.]
 [3. 2. 6. 6. 5.]]


4\. Transformer la matrice de fréquence en PWM. Il faut diviser chaque élément par la somme de ses colonnes.

In [61]:
def generatePWM(MF):
  """
  Transforme la matrice de fréquence en PWM
  entrée MF: matrice de fréquence
  sortie PWM: matrice de probabilité (poids positions)
  """
  tmp = 0
  PWM = np.zeros((len(nuc), k))
  for i in range(len(MF[0])):
      tmp = MF[0][i]  +   MF[1][i] + MF[2][i] + MF[3][i]
      for j in range(len(nuc)):
        PWM[j][i] = np.round(MF[j][i] / tmp, 3)
  return PWM


PWM = generatePWM(MF)
print(PWM)

[[0.4 0.3 0.1 0.  0. ]
 [0.1 0.1 0.1 0.1 0.2]
 [0.2 0.4 0.2 0.3 0.3]
 [0.3 0.2 0.6 0.6 0.5]]


5\. Faire une fonction pour calculer la probabilité d'un motif de taille `k` selon une PWM.


In [62]:
def probability(PWM, motif):
    """
    Calcul la probalité d'un motif selon PWM
    entrée PWM: matrice de probabilité (poids positions)
    entrée motif: motif
    sortie prob: probalité Prob(motif|PWM)
    """
    prob = 1
    i = 0
    for lettre in motif:
        if (i >= len(PWM[0])):
            break
        if lettre.upper() == 'A':
            prob = prob * PWM[0][i] 
        elif lettre.upper() == 'C':
            prob = prob * PWM[1][i] 
        elif lettre.upper() == 'G':
            prob = prob * PWM[2][i]
        elif lettre.upper() == 'T':
            prob = prob * PWM[3][i]
        i = i + 1
    return prob

prob = probability(PWM, "AGgCA")
print (prob)

0.0


6\. Faire une fonction pour calculer le pMostProbkmer d'une séquence, voir un exemple dans les slides de cours.

In [63]:
def pMostProbkmer(PWM, k, sequence):
    """
    calcul la position du k-mer le plus probable dans la séquence
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: la position la plus probable 
    """
    maxProb = 0
    s = 0 # position du k-mer le plus probable dans la sequence
    for i in range(len(sequence) - k + 1):
        tmp = probability(PWM, sequence[i:k + i])
        if (tmp > maxProb):
            s = i
            maxProb = tmp
    return s
s = pMostProbkmer(PWM, k, "GCCGGATACT")
print(s)

5


7\. Faire une fonction pour obtenir les nouvelles positions de départ `s = (s1, …,st)`, c’était à dire les positions qui contiens les k-mer le plus probables.

In [64]:
def getNewS(PWM, k, sequences):
    """
    Trouve les nouvelles positions des k-mer le plus probables
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: vecteur avec les positions les plus problables
    """
   
    return [pMostProbkmer(PWM, k, seq) for seq in sequences]

ns = getNewS(PWM, k, adn)
print (ns)

[5, 3, 1, 1, 5, 0, 0, 0, 0, 0]


8\. La condition d’arrêt de l'algorithme est le non changement de la matrice de fréquence d'une itération à l'autre, 
faire une fonction pour comparer deux matrices et détecter le changement. 
Faire aussi une fonction pour obtenir le score d'une matrice de fréquence : la somme de max de chaque colonne

In [65]:
def changeProfile(P1, P2):
    """
    Compare deux matrice
    entrée P1: matrice de fréquence
    entrée P2: matrice de fréquence
    sortie: True si les matrices sont différents, False au contraire
    """
    return not((P1 == P2).all())
    

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    sc = 0
    t = len(MF)
    n = len(MF[0])
    m = 0
    for i in range(n) :
        m = 0
        for j in range(t) :
            val = MF[j][i]
            if val > m :
                m = val
        sc += m
    
    return sc
ns = getNewS(PWM, k, adn)
print ("s :", s, "\nns :", ns)

sc = getScore(MF, k)
print ("getScore :", sc)

s : 5 
ns : [5, 3, 1, 1, 5, 0, 0, 0, 0, 0]
getScore : 25.0


9\. Implementer l'algorithme ``GreedyProfileMotifSearch`` en utilisant les fonctions precedentes. 
BONUS: Pour ameliorer la performance vous pouvez abandonner les positions qui génèrent des motifs peu complexes.

In [66]:
def GreedyProfileMotifSearch(sequences, k):
    """
    Implémente l'algorithme GreedyProfileMotifSearch 
    entrée sequences: matrice de dimension txn contenant les séquences 
    entrée k : taille du motif 
    sortie s : vecteur de positions de départ ayant le meilleur motif 
    sortie bestScore : le score associé à s
    """
    bestScore = math.inf
    savePWM = np.zeros((len(nuc), k))
    s = generateRandomS(sequences, k) # initialisation de s avec des positions aléatoires
    while (True):
        seqfroms = [seq[s[i]:k+s[i]] for i, seq in enumerate(sequences)]
        p = profile(seqfroms, k, nuc)
        s = []
        bestScore = getScore(p,k)
        PWM = generatePWM(p)
        for seq in sequences:
            s.append(pMostProbkmer(PWM,k,seq))
        if (changeProfile(PWM,savePWM)):
            break
        savePWM = PWM
   
    return s , bestScore


print (adn)
s, bestScore = GreedyProfileMotifSearch(adn, k)   
motifs = extractSeqs(s, adn, k)
print(bestScore)
print (motifs)
        

['CGGAAAGTTG', 'TTAAGTTTTT', 'GAATTTTTGT', 'GAGTTTGAAA', 'GTTAGAGTTT', 'GTTTG', 'TAACG', 'CGGGT', 'TACGC', 'GCTGC']
20.0
['GAAAG', 'TAAGT', 'TTTGT', 'TTTGA', 'TTAGA', 'GTTTG', 'TAACG', 'CGGGT', 'TACGC', 'GCTGC']


10\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

Reponse: on trouve GTT

11\. Vous avez certainement observer que l’algorithme ne produire pas toujours la même sortie et que les résultats dépendent de la sélection aléatoire des positions de départ `s`. Pour augmenter nous chances de retrouvez les bons motifs, nous allons implémenter une version itérative ``GreedyProfileMotifSearchIte`` 
qui a chaque fois sauvegarde `s` et le score du profile associé à `s`, après `I` itération, l’algorithme renvoie le vecteur `s` ayant le plus grand score.

In [67]:
def GreedyProfileMotifSearchIte(sequences, k, It):
    """
    Implémente l'algorithme GreedyProfileMotifSearch iteratif
      entrée séquences: matrice de dimension txn contenant les séquences
      entrée k: taille du motif 
    entrée It: nombre d'iterations
    sortie positions: dictionnaire clé=score, valeur= vecteur s 
    sortie consensus: dictionnaire clé=score, valeur= sequence consensus du motif
    """
   
    pos = {}
    consensus = {}
    
    for i in range(It):
        s,score = GreedyProfileMotifSearch(sequences, k)
        pos[score] = s
        
        kmers = extractSeqs(s, sequences, k)
        profile_var = profile(kmers, k, nuc)
        PWM_var= generatePWM(profile_var)
        
        maxProb = 0
        bestMotif = ""
        for i in kmers :
            proba_var = probability(PWM_var, i)
            if proba_var > maxProb :
                maxProb = proba_var
                bestMotif = i
        
        consensus[score] = bestMotif
        
    
    max_key = max(pos, key=pos.get)
    return pos[max_key], consensus
s, consensus = GreedyProfileMotifSearchIte(adn, k, 50)

print (s)
print (consensus)

[4, 4, 5, 0, 4, 0, 0, 0, 0, 0]
{21.0: 'TAAGT', 19.0: 'GAGTT', 22.0: 'TAAGT', 20.0: 'GAATT', 23.0: 'GTTTT', 24.0: 'TAAGT', 25.0: 'GAATT'}


In [68]:
def printTopFMotifsScore(consensus, top):
    unique_values = list(set(consensus.values()))
    new_dict = {k: v for k, v in consensus.items() if v in unique_values}
    print(new_dict)
    motifsSort = sorted(new_dict.keys())
    motifsSort.reverse()
    for i in range(top):
        print(new_dict[motifsSort[i]])

print(consensus)
printTopFMotifsScore(consensus, 2)


{21.0: 'TAAGT', 19.0: 'GAGTT', 22.0: 'TAAGT', 20.0: 'GAATT', 23.0: 'GTTTT', 24.0: 'TAAGT', 25.0: 'GAATT'}
{21.0: 'TAAGT', 19.0: 'GAGTT', 22.0: 'TAAGT', 20.0: 'GAATT', 23.0: 'GTTTT', 24.0: 'TAAGT', 25.0: 'GAATT'}
GAATT
TAAGT


12\. Tester algorithme  ``GreedyProfileMotifSearchIte`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats.

In [69]:

def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

In [70]:
k=8

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_3.fasta"


sequencesChip   = readFasta(genome, n)
t = len(sequencesChip)
#print (sequencesChip[8], t, k)
revSequences = [reverseComplement(m) for m in sequencesChip]
sequencesChip = sequencesChip + revSequences
sequencesRet = [x for x in sequencesChip if len(x) == n]
#print(removeLowComplex(sequencesRet))
s, consensus = GreedyProfileMotifSearchIte(sequencesRet, k, 100)
print(consensus)
printTopFMotifsScore(consensus, 8)


{22190.0: 'ATTATATT', 22054.0: 'ATTTATAT', 22001.0: 'ATAGTATT', 22081.0: 'TATTATAT', 22104.0: 'AATAATAT', 22132.0: 'TATATTAT', 22167.0: 'TATTATAT', 22055.0: 'AAATTAAT', 22142.0: 'ATAAATAT', 22150.0: 'AATATTTA', 22179.0: 'AAATAATT', 22153.0: 'AATAATAT', 22049.0: 'AATATCTT', 21940.0: 'AATATATA', 22158.0: 'AATAAATA', 22147.0: 'TAAATATT', 22087.0: 'TATTATTA', 22074.0: 'AATATATA', 22050.0: 'ATGTACTT', 22098.0: 'TATTTAAT', 22070.0: 'TATATAAT', 22004.0: 'ATTGTATT', 22009.0: 'AATAAATA', 22035.0: 'AATTATTA', 22053.0: 'ATTTTATT', 22066.0: 'AATTATTT', 22127.0: 'TAATAAAT', 22157.0: 'ATATAAAT', 21986.0: 'AATCTAAT', 21971.0: 'ATTATATA', 22100.0: 'ATATATTT', 22110.0: 'TAATAATA', 22039.0: 'ATATTATT', 22120.0: 'AATCTATT', 22082.0: 'TATGTATT', 21998.0: 'ATAATATT', 22109.0: 'TATTTATT', 22148.0: 'AAGTAATA', 22089.0: 'AATAATAT', 22056.0: 'ATATATTT', 22093.0: 'TATATATA', 22079.0: 'ATAATATT', 22016.0: 'AATATATT', 22013.0: 'TAATATAT', 22042.0: 'TATATAAT', 22017.0: 'ATATAATA', 22141.0: 'AATATAAT', 22067.0: 'AT